Hunter Bennett | Glass Lab | Kupffer Strains Project | February 24 2021  

This notebook is for making fancier figure level plots describing differential gene expression for the strains NASH paper. It generates the following plots:
1. Differential scatter plot similar to those make in Link et al. 2018 showing differential peaks at fc > 2 and fc > 4.

In [13]:
### header ###
__author__ = "Hunter Bennett"
__license__ = "BSD"
__email__ = "hunter.r.bennett@gmail.com"
%load_ext autoreload
%autoreload 2
### imports ###
import sys
%matplotlib inline
import os
import re
import glob
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 300
sns.set(font_scale=1)
sns.set_context('talk')
sns.set_style('ticks')

# import custom functions
import sys
sys.path.insert(0, '/home/h1bennet/code/')
from hbUtils import ngs_qc, quantile_normalize_df
from plotting_scripts import label_point, pca_rpkm_mat, get_diff_volcano
from homer_preprocessing import read_annotated_peaks, import_homer_diffpeak, pull_comparisons_get_diff

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
dataDirectory = ''
workingDirectory = '/home/h1bennet/strains/results/00_Strains_Control_ATAC/'
if not os.path.isdir(workingDirectory):
    os.mkdir(workingDirectory)
os.chdir(workingDirectory)

# Make folder for "fancy figures" for manuscript

In [15]:
if not os.path.isdir('./fancy_figures/'):
    os.mkdir('./fancy_figures/')

### Import differential peaks

In [16]:
diff_peak, peaks, peak_mat, peak_mat_quant = import_homer_diffpeak(
    './merged_peaks/diff_output.txt',
    './merged_peaks/ann_norm_idr_peaks_merged.txt')

annotatePeaks all peaks (64770, 26)
getDiffExpression selected transcripts (63491, 35)
annotatePeaks selected peaks (63491, 26)


In [17]:
comp_dict = pull_comparisons_get_diff(diff_peak, seq_type='Peak')

# Set parameters that don't change throughout plotting

In [18]:
comps = ['00_c57 vs. 01_balbc',
         '00_c57 vs. 02_aj',
         '01_balbc vs. 02_aj']

labels = [['C57BL/6J\nlog2 (tag counts + 1)', 'BALB/cJ\nlog2 (tag counts + 1)'],
          ['C57BL/6J\nlog2 (tag counts + 1)', 'AJ\nlog2 (tag counts + 1)'],
          ['BALB/cJ\nlog2 (tag counts + 1)', 'AJ\nlog2 (tag counts + 1)']]

groups = [[[0,1],[2,3,4]],
         [[0,1],[5,6,7]],
         [[2,3,4],[5,6,7]]]

cols = []
for i in groups:
    subcols = []
    for j in i:
        subcols.extend([k for k in j])
    cols.append(subcols)

# set plotting thresholds
pthresh = 0.05
lfcthresh = (1,2)
dotsize = 3
ax_max = 16
n_yticks = 5
ytickstep = 3

# Strain specific coloring

In [19]:
colors = [['#636363', '#6baed6', '#3182bd', '#74c476', '#31a354'],
          ['#636363', '#fb6a4a', '#de2d26', '#74c476', '#31a354'],
          ['#636363', '#fb6a4a', '#de2d26', '#6baed6', '#3182bd']]

filename='h3k27ac_diffpeak_scatter_twothresh_sixcolor.png'

In [20]:
fig, axs = plt.subplots(1,3, figsize=(16,4))
plt.subplots_adjust(wspace=0.5)

for key, col, group, labs, color, ax in zip(comps, cols, groups, labels, colors, axs.reshape(-1)):
    
    de = comp_dict[key]
    de['logtpm'] = np.log2(peak_mat.iloc[:, col].mean(1)+1)
    de['log10p'] = -np.log10(de.adj_pval + 10**(-50))
    de['g0_mean'] = np.log2(peak_mat.iloc[:, group[0]].mean(1)+1)
    de['g1_mean'] = np.log2(peak_mat.iloc[:, group[1]].mean(1)+1)
    
    # plot group by group scatter:
    de_nonsig = de.reindex(de.index[de.adj_pval >= pthresh])
    de_sig_up_fc1 = de.reindex(de.index[(de.adj_pval < pthresh) & (de.log2fc >= lfcthresh[0])])
    de_sig_up_fc2 = de.reindex(de.index[(de.adj_pval < pthresh) & (de.log2fc >= lfcthresh[1])])
    de_sig_dn_fc1 = de.reindex(de.index[(de.adj_pval < pthresh) & (de.log2fc <= -lfcthresh[0])])
    de_sig_dn_fc2 = de.reindex(de.index[(de.adj_pval < pthresh) & (de.log2fc <= -lfcthresh[1])])
    
    for df, c, in zip([de_nonsig, de_sig_up_fc1, de_sig_up_fc2, de_sig_dn_fc1, de_sig_dn_fc2],
                      color):
        ax.scatter(
            x=df.g0_mean,
            y=df.g1_mean,
            c=c,
            s=dotsize)
    
    # set axis labels
    ax.set_xlabel(labs[0])
    ax.set_ylabel(labs[1])
    
    #set max and min
    ax.set_xlim([0,ax_max])
    ax.set_ylim([0,ax_max])
    
    # set ticks
    ax.set_xticks(np.arange(0,ax_max,ytickstep))
    ax.set_xticklabels(np.arange(0,ax_max,ytickstep))
    ax.set_yticks(np.arange(0,ax_max,ytickstep))
    ax.set_yticklabels(np.arange(0,ax_max,ytickstep))
    
    
plt.savefig('./fancy_figures/'+filename, bbox_inches='tight')
plt.close()

# Non-specific coloring

In [21]:
colors = [['#636363', '#fb6a4a', '#de2d26'],
          ['#636363', '#fb6a4a', '#de2d26'],
          ['#636363', '#fb6a4a', '#de2d26']]
    
filename='h3k27ac_diffpeak_scatter_twothresh_twocolor.png'

In [22]:
fig, axs = plt.subplots(1,3, figsize=(16,4))
plt.subplots_adjust(wspace=0.5)

for key, col, group, labs, color, ax in zip(comps, cols, groups, labels, colors, axs.reshape(-1)):
    
    de = comp_dict[key]
    de['logtpm'] = np.log2(peak_mat.iloc[:, col].mean(1)+1)
    de['log10p'] = -np.log10(de.adj_pval + 10**(-50))
    de['g0_mean'] = np.log2(peak_mat.iloc[:, group[0]].mean(1)+1)
    de['g1_mean'] = np.log2(peak_mat.iloc[:, group[1]].mean(1)+1)
       
    # plot group by group scatter:
    de_nonsig = de.reindex(de.index[de.adj_pval >= pthresh])
    de_sig_fc1 = de.reindex(de.index[(de.adj_pval < pthresh) & (np.abs(de.log2fc) >= lfcthresh[0])])
    de_sig_fc2 = de.reindex(de.index[(de.adj_pval < pthresh) & (np.abs(de.log2fc) >= lfcthresh[1])])
    
    for df, c in zip([de_nonsig, de_sig_fc1, de_sig_fc2],
                     color):
        ax.scatter(
            x=df.g0_mean,
            y=df.g1_mean,
            c=c,
            s=dotsize)
     
    # set axis labels
    ax.set_xlabel(labs[0])
    ax.set_ylabel(labs[1])
    
    #set max and min
    ax.set_xlim([0,ax_max])
    ax.set_ylim([0,ax_max])
    
    # set ticks
    ax.set_xticks(np.arange(0,ax_max,ytickstep))
    ax.set_xticklabels(np.arange(0,ax_max,ytickstep))
    ax.set_yticks(np.arange(0,ax_max,ytickstep))
    ax.set_yticklabels(np.arange(0,ax_max,ytickstep))
    
plt.savefig('./fancy_figures/'+filename, bbox_inches='tight')
plt.close()

# One logfc threshold

In [23]:
colors = [['#636363', '#de2d26'],
          ['#636363', '#de2d26'],
          ['#636363', '#de2d26']]

filename='h3k27ac_diffpeak_scatter_onethresh_onecolor.png'

In [24]:
fig, axs = plt.subplots(1,3, figsize=(16,4))
plt.subplots_adjust(wspace=0.5)

for key, col, group, labs, color, ax in zip(comps, cols, groups, labels, colors, axs.reshape(-1)):

    
    de = comp_dict[key]
    de['logtpm'] = np.log2(peak_mat.iloc[:, col].mean(1)+1)
    de['log10p'] = -np.log10(de.adj_pval + 10**(-50))
    de['g0_mean'] = np.log2(peak_mat.iloc[:, group[0]].mean(1)+1)
    de['g1_mean'] = np.log2(peak_mat.iloc[:, group[1]].mean(1)+1)
       
    # plot group by group scatter:
    de_nonsig = de.reindex(de.index[de.adj_pval >= pthresh])
    de_sig_fc1 = de.reindex(de.index[(de.adj_pval < pthresh) & (np.abs(de.log2fc) >= lfcthresh[0])])
    
    for df, c in zip([de_nonsig, de_sig_fc1],
                     color):
        ax.scatter(
            x=df.g0_mean,
            y=df.g1_mean,
            c=c,
            s=dotsize)
     
    # set axis labels
    ax.set_xlabel(labs[0])
    ax.set_ylabel(labs[1])
    
    #set max and min
    ax.set_xlim([0,ax_max])
    ax.set_ylim([0,ax_max])
    
    # set ticks
    ax.set_xticks(np.arange(0,ax_max,ytickstep))
    ax.set_xticklabels(np.arange(0,ax_max,ytickstep))
    ax.set_yticks(np.arange(0,ax_max,ytickstep))
    ax.set_yticklabels(np.arange(0,ax_max,ytickstep))
      
plt.savefig('./fancy_figures/'+filename, bbox_inches='tight')
plt.close()